In [1]:
from glob import glob
import json

In [2]:
files = glob('texts_5.jsonl*.splitted.requested')
files

['texts_5.jsonl04.splitted.requested',
 'texts_5.jsonl07.splitted.requested',
 'texts_5.jsonl03.splitted.requested',
 'texts_5.jsonl00.splitted.requested',
 'texts_5.jsonl08.splitted.requested',
 'texts_5.jsonl05.splitted.requested',
 'texts_5.jsonl01.splitted.requested',
 'texts_5.jsonl02.splitted.requested',
 'texts_5.jsonl06.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[l['src']] = l['r']['result']
            
len(mapping)

871960

In [4]:
all_data = []
with open('train_5.jsonl') as fopen:
    for l in fopen:
        try:
            l = json.loads(l)
            data = []
            for d_ in l['data']:
                data.append({
                    'value': d_,
                    'value_ms': mapping.get(d_)
                })
            all_data.append(data)
        except:
            pass

In [5]:
len(all_data)

150000

In [6]:
with open('train_5.jsonl.translated', 'w') as fopen:
    for d in all_data:
        fopen.write(f'{json.dumps(d)}\n')

In [7]:
inside = []
with open('train_5.jsonl.translated') as fopen:
    for l in fopen:
        if 'model bahasa' in l:
            inside.append(l)
            if len(inside) >= 10:
                break

In [8]:
break_at = [
    'help.openai.com',
]

In [9]:
from tqdm import tqdm

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        if d['value_ms'] is None:
            break
        if no % 2 == 0:
            t = '<manusia>: '
        else:
            t = '<bot>: '
        
        if any([b in d['value_ms'].lower() for b in break_at]):
            print(c['value_ms'], '\n-----\n')
            break
        
        n = d['value_ms']
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 150000/150000 [00:01<00:00, 98305.26it/s]


In [10]:
len(all_texts)

127970

In [11]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [12]:
with open('part5_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')